[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_toolbox/pdftools_toolbox_encrypt.ipynb)

In [ ]:
%pip install pdftools_toolbox
%pip install ipython

# Encrypt PDF
Encrypt a PDF document with a user password and an owner
password. When opening the document, either of the
passwords must be provided. If providing the user
password, the document can be viewed and printed only.
Providing the owner password grants full access to the
document.

In [ ]:
import io
from pdftools_toolbox.pdf import Document, Encryption, FileReference, Metadata, PageCopyOptions, PageList, Permission
from pdftools_toolbox.pdf.content import IccBasedColorSpace
from pdftools_toolbox.pdf.navigation import ViewerSettings

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/InvoiceNone.pdf'
input_file_path = 'InvoiceNone.pdf'
prepare_file(input_url, input_file_path)
user_pwd = "userpass"
owner_pwd = "ownerpass"
output_file_path = 'EncryptedInvoice.pdf'

In [ ]:
def copy_document_data(in_doc: Document, out_doc: Document):
    # Copy document-wide data

    # Output intent
    if in_doc.output_intent is not None:
        in_doc.output_intent = IccBasedColorSpace.copy(out_doc, in_doc.output_intent)

    # Metadata
    out_doc.metadata = Metadata.copy(out_doc, in_doc.metadata)

    # Viewer settings
    out_doc.viewer_settings = ViewerSettings.copy(out_doc, in_doc.viewer_settings)

    # Associated files (for PDF/A-3 and PDF 2.0 only)
    outAssociatedFiles = out_doc.associated_files
    for in_file_ref in in_doc.associated_files:
        outAssociatedFiles.append(FileReference.copy(out_doc, in_file_ref))

    # Plain embedded files
    out_embedded_files = out_doc.plain_embedded_files
    for in_file_ref in in_doc.plain_embedded_files:
        out_embedded_files.append(FileReference.copy(out_doc, in_file_ref))

In [ ]:
try:
    # Set and check license key. If the license key is not valid, an exception is thrown.
    from pdftools_toolbox.sdk import Sdk
    Sdk.initialize("INSERT-LICENSE-KEY", None)

    # Open input document
    with io.FileIO(input_file_path, "rb") as in_stream:
        with Document.open(in_stream, None) as in_doc:
    
            # Create encryption parameters
            encryption_params = Encryption(
                user_pwd,
                owner_pwd,
                Permission.PRINT | Permission.DIGITAL_PRINT,
            )
    
            # Create output document and set a user and owner password
            with io.FileIO(output_file_path, "wb+") as out_stream:
                with Document.create(out_stream, in_doc.conformance, encryption_params) as out_doc:
    
                    # Copy document-wide data
                    copy_document_data(in_doc, out_doc)
    
                    # Define page copy options
                    copy_options = PageCopyOptions()
    
                    # Copy all pages and append to output document
                    copied_pages = PageList.copy(out_doc, in_doc.pages, copy_options)
                    out_doc.pages.extend(copied_pages)

    print("Execution successful.")
except Exception as e:
    print(f"An error occurred: {e}")